In [ ]:
import glob
import os
import librosa
from librosa import display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')


plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [ ]:
features = np.loadtxt('nn_simple_features.csv', delimiter=',')
labels = np.array(np.loadtxt('nn_simple_labels.csv', delimiter=','), dtype=np.int)

In [ ]:
from sklearn.preprocessing import normalize

X_all = features
y_all = labels

In [143]:
from sklearn.model_selection import train_test_split
import time

X_not_test, X_test, y_not_rest, y_test = train_test_split(
    X_all, y_all, stratify=y_all, train_size=.85, random_state=round(time.time()))

X_train, X_val, y_train, y_val = train_test_split(
    X_not_test, y_not_rest, stratify=y_not_rest, train_size=.9, random_state=round(time.time()))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [144]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(9046, 1280)
(9046, 5)
(1006, 1280)
(1006, 5)
(1774, 1280)
(1774, 5)


In [ ]:
## Imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
# print(K.image_data_format())

# X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
# X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
# X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
# input_shape = (img_rows, img_cols, 1)
# print(X_train.shape)
# print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
# print(X_test.shape)
# print(y_test.shape)

In [145]:
# Parameters
# learning_rate = 0.005

global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.005
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           50, 0.99, staircase=True)

training_iters = 4000
batch_size = 100
test_step = 40

# Network Parameters
n_input = 40 * 32
n_classes = 5
dropout = .8 # Dropout, probability to keep units

# 1. Define Variables and Placeholders
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

def build_model(x, dropout, activation):
    
    x = tf.reshape(x, shape=[-1, 40, 32, 1])
    
    conv1 = tf.layers.conv2d(x, 4, 5, activation=activation)
    conv1 = tf.layers.batch_normalization(conv1)
    conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
    conv1 = tf.nn.dropout(conv1, dropout)
    
    conv2 = tf.layers.conv2d(conv1, 8, 3, activation=activation)
    conv2 = tf.layers.batch_normalization(conv2)
    conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
    conv2 = tf.nn.dropout(conv2, dropout)
    
    fc1 = tf.contrib.layers.flatten(conv2)

    fc1 = tf.layers.dense(fc1, 128, activation=activation)
    fc1 = tf.layers.batch_normalization(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    fc2 = tf.layers.dense(fc1, 64, activation=activation)
    fc2 = tf.layers.batch_normalization(fc2)
    fc2 = tf.nn.dropout(fc2, dropout)
    
    out = tf.layers.dense(fc2, n_classes)

    return out

predictions = build_model(x, keep_prob, activation=tf.nn.relu)
# 3. Define the loss function  
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
# 4. Define the accuracy 
correct_pred = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 5. Define an optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost, global_step=global_step)

# Initializing the variables
init = tf.global_variables_initializer()

In [146]:
train_accuracy = []
train_cost = []
val_accuracy = []
val_cost = []

def feed_next_batch(train_size, batch_size=64):
    
    start = 0
    while start < train_size:
        yield start, start + batch_size
        start += batch_size

with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step < training_iters:
        
        indices = np.arange(len(y_train))
        np.random.shuffle(indices)
        X_train, y_train =  X_train[indices], y_train[indices]
        
        for start, end in feed_next_batch(len(X_train), batch_size=batch_size):
            # Run optimization op (backprop)
            
            batch_x, batch_y = X_train[start:end], y_train[start:end]
            
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            if step % test_step == 0:
                # Calculate batch loss and accuracy
                train_loss, train_acc = sess.run([cost, accuracy], feed_dict={x: X_train, 
                                                                              y: y_train,
                                                                              keep_prob: 1.0})

                if type(learning_rate) is not float:
                    print('learning_rate:', sess.run(learning_rate))

                train_cost.append(train_loss)
                train_accuracy.append(train_acc)

                print ("Batch " + str(step) + ", Training Loss= " + \
                      "{:.6f}".format(train_loss) + ", Train Accuracy= " + \
                      "{:.5f}".format(train_acc))

                val_loss, val_acc = sess.run([cost, accuracy], feed_dict={x: X_val, 
                                                                            y: y_val,
                                                                           keep_prob: 1.0})
                val_cost.append(val_loss)
                val_accuracy.append(val_acc)
                print ("Validation loss:", val_loss, ', Validation acc: ', val_acc)
                print('-' * 80)

            step += 1
            
    print ("Optimization Finished!")

    # Calculate accuracy for all test samples
    print ("Training Accuracy:", \
        sess.run(accuracy, feed_dict={x: X_train,
                                      y: y_train,
                                     keep_prob: 1.0}))
    print ("Test Accuracy:", \
        sess.run(accuracy, feed_dict={x: X_test,
                                      y: y_test,
                                     keep_prob: 1.0}))

learning_rate: 0.005
Batch 40, Training Loss= 1.598171, Train Accuracy= 0.26631
Validation loss: 1.60151 , Validation acc:  0.261431
--------------------------------------------------------------------------------
learning_rate: 0.00495
Batch 80, Training Loss= 1.528448, Train Accuracy= 0.31793
Validation loss: 1.53208 , Validation acc:  0.316103
--------------------------------------------------------------------------------
learning_rate: 0.0049005
Batch 120, Training Loss= 1.441624, Train Accuracy= 0.38017
Validation loss: 1.45431 , Validation acc:  0.371769
--------------------------------------------------------------------------------
learning_rate: 0.00485149
Batch 160, Training Loss= 1.361797, Train Accuracy= 0.44075
Validation loss: 1.39179 , Validation acc:  0.43837
--------------------------------------------------------------------------------
learning_rate: 0.00480298
Batch 200, Training Loss= 1.259116, Train Accuracy= 0.51437
Validation loss: 1.28202 , Validation acc:  0.